# Machine Learning beadandó feladata
## Dataracing 2022: exportforgalom előrejelzés
Király Márk (AX83OL)

### Importáljuk a szükséges modulokat.

In [4]:
import pandas as pd
import optuna
from optuna.samplers import TPESampler, GPSampler
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from imblearn.over_sampling import SMOTE
import numpy as np

### Töltsük be az adatokat az alábbi sorrendben:
#### 1. csv betöltése, 2. célváltozó beállítása, 3. tanító-teszt 0,6-0,4 felosztás
Megj.: Ez a kommentek javítása miatt lett később lefuttatva, mint a többi cella => ha újból futtatnánk az összes cellát, akkor is hasonló MAE, MSE, R2 értékeket kapnánk.

In [16]:
# Adatok betöltése
df = pd.read_csv("https://raw.githubusercontent.com/karsarobert/Machine_Learning_2024/main/train.csv")

# Célváltozó és prediktorok beállítása
y = df['target_reg']
corr_col = ['arbevexp_2014', 'arbevexp_2015', 'arbevexp_2016', 'arbevert_2014', 'arbevert_2015', 'arbevert_2016']
X = df[corr_col]

# Adatok felosztása tanító és teszt halmazokra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

### Végezzünk adatbővítést! (Zajjal és szintetikus adatok hozzáadásával is)
#### Zajt adunk hozzá 0,1-es faktorral és 2-szer annyi szintetikus adatot gyártunk lineáris kombinációval, mint az eredeti tanítóhalmaz
#### Ok: a tanítóhalmaz túl kicsi a 0,6-0,4-es felosztás miatt...

In [17]:
# Zaj hozzáadása
noise_factor = 0.1
X_train_noisy = X_train + noise_factor * np.random.randn(*X_train.shape)
X_test_noisy = X_test + noise_factor * np.random.randn(*X_test.shape)

# Szintetikus adatok generálása lineáris kombinációval
def generate_synthetic_data(X, y, num_samples):
    synthetic_X = []
    synthetic_y = []
    num_features = X.shape[1]
    
    for _ in range(num_samples):
        idx1, idx2 = np.random.choice(len(X), size=2, replace=False)
        alpha = np.random.rand()
        
        new_sample_X = alpha * X.iloc[idx1] + (1 - alpha) * X.iloc[idx2]
        new_sample_y = alpha * y.iloc[idx1] + (1 - alpha) * y.iloc[idx2]
        
        synthetic_X.append(new_sample_X)
        synthetic_y.append(new_sample_y)
    
    return pd.DataFrame(synthetic_X, columns=X.columns), pd.Series(synthetic_y)

# Generáljunk szintetikus adatokat a tanító halmazhoz
num_synthetic_samples = len(X_train) * 2  # 2-szer több szintetikus minta, mint az eredeti tanító halmaz
X_synthetic, y_synthetic = generate_synthetic_data(X_train, y_train, num_synthetic_samples)

# Kombináljuk az eredeti és szintetikus adatokat
X_train_augmented = pd.concat([X_train_noisy, X_synthetic])
y_train_augmented = pd.concat([y_train, y_synthetic])

# StandardScaler létrehozása és illesztése a tanító adatokra
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_augmented)
X_test_scaled = scaler.transform(X_test_noisy)

# Az újonnan zajosított és skálázott adatok ellenőrzése
print("X_train_scaled (augmented):", X_train_scaled[:5])
print("X_test_scaled:", X_test_scaled[:5])

X_train_scaled (augmented): [[-0.11330591 -0.10912084 -0.10768715 -0.00083785 -0.03849117 -0.1954651 ]
 [-0.11330586 -0.10912077 -0.10768718 -0.23330005 -0.23690373 -0.23116736]
 [-0.11330598 -0.10912076 -0.10768725 -0.099985   -0.11537316 -0.1158518 ]
 [-0.11330588 -0.10912069 -0.10768722 -0.24661754 -0.24468327 -0.23788795]
 [-0.11330599 -0.1091208  -0.10768711 -0.2435801  -0.2433832  -0.23787602]]
X_test_scaled: [[-0.11330601 -0.10912084 -0.10768718 -0.23302266 -0.23560083 -0.23243503]
 [-0.11330573 -0.10912071 -0.10768712  1.41922932  1.50132954  1.3707072 ]
 [-0.11330592 -0.10912068 -0.10768724 -0.18437169 -0.18995129 -0.19806263]
 [ 0.0255127   0.00175727 -0.00856693 -0.15829477 -0.17811143 -0.16801288]
 [-0.11330582 -0.10912067 -0.10768712 -0.22912825 -0.22927216 -0.22774911]]


## Építsünk modelt!
### Futtatáskor figyeljünk rá, hogy a MAE-t és a további jellemzőket az eredeti halmazon mérjük (y_test, y_pred).
### A modell a következőket használja: 
#### Random Forest regresszió, 
#### Optuna hiperparaméterkereső keretrendszer, ezen belűl a TPESampler fa alapú keresőalgoritmus

In [18]:
# Célfüggvény definiálása
def objective(trial):
    # Hiperparaméterek meghatározása
    n_estimators = trial.suggest_int('n_estimators', 10, 200) #(10,200)
    max_depth = trial.suggest_int('max_depth', 1, 20) #(1,20)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 10) #(2,10)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 4) #(1,4)
    
    # Random Forest modell létrehozása a megadott hiperparaméterekkel
    model = RandomForestRegressor(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        random_state=42,
        n_jobs=-1
    )
    
    # Modell illesztése
    model.fit(X_train_scaled, y_train_augmented)
    
    # Előrejelzések készítése
    y_pred = model.predict(X_test_scaled)
    
    # MAE számolása
    mae = mean_absolute_error(y_test, y_pred)
    return mae

# "Tanulmány" létrehozása és futtatása
try:
    study = optuna.create_study(direction='minimize', sampler=TPESampler())
    study.optimize(objective, n_trials=100)  # , timeout=900) Tehát maximum 0,25 óráig fut - kikommentelve
except optuna.exceptions.TrialPruned as e:
    print("Optimization stopped:", e)

# Legjobb paraméterek kiíratása
print("Best hyperparameters: ", study.best_params)

# Legjobb modell létrehozása a legjobb paraméterekkel
best_params = study.best_params
best_model = RandomForestRegressor(
    n_estimators=best_params['n_estimators'],
    max_depth=best_params['max_depth'],
    min_samples_split=best_params['min_samples_split'],
    min_samples_leaf=best_params['min_samples_leaf'],
    random_state=42,  # Marad változatlan!
    n_jobs=-1  # Párhuzamosítás miatt kell (igaz csak CPU-n párhuzamosít...)
)

# Legjobb modell illesztése a teljes edzési adatokra
best_model.fit(X_train_scaled, y_train_augmented)

# Előrejelzések készítése a teszthalmazon
y_pred = best_model.predict(X_test_scaled)

# Hibametrikák kiértékelése és kiíratása
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
print('Final Model Performance on Test Set:')
print('Mean Squared Error:', mse)
print('R-squared Score:', r2)
print('Mean Absolute Error:', mae)

[I 2024-06-02 10:04:59,566] A new study created in memory with name: no-name-e6fb5804-bbd8-40dc-af1c-43b637d8cfac
[I 2024-06-02 10:05:04,680] Trial 0 finished with value: 48291.54805187346 and parameters: {'n_estimators': 90, 'max_depth': 19, 'min_samples_split': 3, 'min_samples_leaf': 3}. Best is trial 0 with value: 48291.54805187346.
[I 2024-06-02 10:05:06,017] Trial 1 finished with value: 66400.78307222281 and parameters: {'n_estimators': 93, 'max_depth': 4, 'min_samples_split': 10, 'min_samples_leaf': 3}. Best is trial 0 with value: 48291.54805187346.
[I 2024-06-02 10:05:14,728] Trial 2 finished with value: 48403.28179274183 and parameters: {'n_estimators': 196, 'max_depth': 13, 'min_samples_split': 10, 'min_samples_leaf': 3}. Best is trial 0 with value: 48291.54805187346.
[I 2024-06-02 10:05:16,377] Trial 3 finished with value: 49573.78651163134 and parameters: {'n_estimators': 45, 'max_depth': 11, 'min_samples_split': 7, 'min_samples_leaf': 4}. Best is trial 0 with value: 48291.5

Best hyperparameters:  {'n_estimators': 87, 'max_depth': 14, 'min_samples_split': 8, 'min_samples_leaf': 2}
Final Model Performance on Test Set:
Mean Squared Error: 178867436035.9132
R-squared Score: 0.9708414213562483
Mean Absolute Error: 47805.93572784237


Best hyperparameters:  {'n_estimators': 87, 'max_depth': 14, 'min_samples_split': 8, 'min_samples_leaf': 2}
Final Model Performance on Test Set:
Mean Squared Error: 178867436035.9132
R-squared Score: 0.9708414213562483
Mean Absolute Error: 47805.93572784237

## MAE: 47805